<a href="https://colab.research.google.com/github/sarjakpatel/CMPE_297_Advanced_Deep_Learning/blob/main/Assignment%202/CMPE_297_Assignment_2_Part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CMPE 297 Assignment 2 Part 3: Link Prediction for Graph

## Importing Libraries

In [15]:
!pip install torch_geometric

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [16]:
!pip install torch_sparse

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip install torch_scatter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Initialize dataset and Dataset Transformation

In [20]:
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])

In [21]:
dataset = 'Cora'
path = osp.join('.', 'data', dataset)
dataset = Planetoid(path, name='Cora', transform=transform)

After applying the `RandomLinkSplit` transform, the data is transformed from
a data object to a list of tuples (train_data, val_data, test_data), with
each element representing the corresponding split.

In [22]:
train_data, val_data, test_data = dataset[0]

## Model for Link Prediction

In [23]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()

In [24]:
model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()

## Trainning a Model

In [25]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss

## Testing a Model

In [26]:
@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

Epoch: 001, Loss: 0.6930, Val: 0.7195, Test: 0.7095
Epoch: 002, Loss: 0.6829, Val: 0.7151, Test: 0.7064
Epoch: 003, Loss: 0.7043, Val: 0.7130, Test: 0.7157
Epoch: 004, Loss: 0.6781, Val: 0.7152, Test: 0.7265
Epoch: 005, Loss: 0.6848, Val: 0.7207, Test: 0.7322
Epoch: 006, Loss: 0.6879, Val: 0.7258, Test: 0.7356
Epoch: 007, Loss: 0.6889, Val: 0.7288, Test: 0.7375
Epoch: 008, Loss: 0.6884, Val: 0.7307, Test: 0.7409
Epoch: 009, Loss: 0.6865, Val: 0.7271, Test: 0.7391
Epoch: 010, Loss: 0.6823, Val: 0.7217, Test: 0.7370
Epoch: 011, Loss: 0.6749, Val: 0.7187, Test: 0.7343
Epoch: 012, Loss: 0.6673, Val: 0.7162, Test: 0.7327
Epoch: 013, Loss: 0.6690, Val: 0.7218, Test: 0.7449
Epoch: 014, Loss: 0.6631, Val: 0.7403, Test: 0.7759
Epoch: 015, Loss: 0.6514, Val: 0.7481, Test: 0.7945
Epoch: 016, Loss: 0.6420, Val: 0.7355, Test: 0.7946
Epoch: 017, Loss: 0.6272, Val: 0.7210, Test: 0.7834
Epoch: 018, Loss: 0.6133, Val: 0.7112, Test: 0.7764
Epoch: 019, Loss: 0.5988, Val: 0.7164, Test: 0.7809
Epoch: 020, 

## Model Evaluation

In [27]:
print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Final Test: 0.9100


## References



*   http://cs230.stanford.edu/projects_spring_2020/reports/38854344.pdf




*   https://colab.research.google.com/drive/1f2KVSVsk5f4dpZlxNE0RBIHowqDs3KSp#scrollTo=dQ98qUOAt5JK



*   https://github.com/pyg-team/pytorch_geometric/blob/master/examples/link_pred.py
